In [13]:
import os # this allows simple navigation of folders in the notebook
import pandas as pd # this loads Pandas
import numpy as np # This loads NumPy, used to do maths in the notebook
from ebmdatalab import bq # this loads the bennett institute bigquery function - probably not needed in this instance 

In [14]:
importfile = os.path.join("..","data","ppi_data.csv.xz") # creates the importfile for the data
#exportfile = os.path.join("..","data","ppi_data.csv.xz") # creates the importfile for the data

In [15]:
ppi_df = pd.read_csv(importfile) # opens the csv in the zip file, and loads into into a dataframe called "ppi_df"

In [16]:
ppi_df.head() # shows a sample of the first 5 rows of data

,month,practice,bnf_code,bnf_name,items,actual_cost,quantity
0,2023-08-01 00:00:00 UTC,A81002,0103050L0AAAGAG,Lansoprazole 15mg orodispersible tablets,24,78.52231,812
1,2023-08-01 00:00:00 UTC,A81007,0103050P0AAAAAA,Omeprazole 20mg gastro-resistant capsules,415,1680.22668,17598
2,2023-08-01 00:00:00 UTC,A81007,0103050L0AAABAB,Lansoprazole 15mg gastro-resistant capsules,345,391.11221,12349
3,2023-08-01 00:00:00 UTC,A81011,0103050L0AAAAAA,Lansoprazole 30mg gastro-resistant capsules,685,844.09149,20782
4,2023-08-01 00:00:00 UTC,A81011,0103010H0AAABAB,Famotidine 40mg tablets,22,712.78739,704
